# Basics of simulation based computational neuroscience


This notebook aims to explain the principles and techniques of simulation-based network neuroscience. Simulation-based insights are particularly helpful in a system where you cannot derive insights analytically due to the differential equations being non-solvable or including non-linearities. Such a non-linearity is e.g. the reset of the membrane potential in simple spiking neuron models. 

In neuroscience we want to model neural behaviour and have a plethora of neuron models to chose from to do so. They vary in complexity and biological plausibility. In general it is true that the higher the complexity of the model the more computational resources are needed for simulating the system. Therefore, biological realism comes at the cost of computational effeciency. 

The computationally most effecient models are rate-based neuron models. An added bonus of them is, that they can be analysed analytically. The rate-based workshop is giving you some more information and insight into this. 

For neuron models which are defined by non-solvable differential equations like the simple Leaky-Integrate and Fire (LIF) neuron or the Hodgkin-Huxley neuron model, we need to use numerical integration to solve the equation. There are different numerical integration methods and in the following we will discuss the Euler method which is also used by simulation software such as BRIAN.


# Numerical Integration: The Euler method

Models in computational neuroscience are predominantely based on **(Ordinary) Differential Equations** (ODEs). Differential equations define the change of a system's state with regard to a certain quantity. In neuroscience, we often want to understand how the state of a neuron changes over time given an external stimulus or neuromodulation. Therefore neural ODEs or neural models define the change of neural activity $x$ (e.g. the rate or the membrane potential) over time: 
\begin{align*}\frac{dx}{dt} = f(x(t))\end{align*}. 
Here $x(t)$ corresponds to the neural state at time $t$ and $f(x(t))$ defines how the neural activity changes over the integration time step $t$. 

Given the differential equations of this form, we can simulate how the neural activity develops given an initial value $x(t=0)$ (also called the initial condition). We do this by calculating the successive neural states starting with our intial condition: 
\begin{align*} x(t=1) &= x(t=0) + f(x(t=0)) \\ x(t=2) &= x(t=1) + f(x(t=1)) \\ &... \\ x(t=T) &= x(t=T-1) + f(x(t=T-1))\end{align*}
with $T$ being the last timestep we want to simulate. Formally the Euler method can be thus defined as 
\begin{align*} x(t+dt) = x(t) + \frac{dx}{dt} \end{align*}. 
$dt$ is the integration time step. In other words the temporal resolution we use to simulate the system. The smaller the integration time step $dt$, the smaller the error between the "true value" of $x(t+dt)$ and the approximation $x(t) + \frac{dx}{dt}$.

The graph below shows the Euler method graphically. In this case we know the true form of $x(t)$ at each point. Therefore, we can that if $dt$ is big the discrepancy between the true value and the approximation via the Euler method is big as well. Therefore, $dt$ should always be small. For neuron models $dt$= 0.1ms is a standard value. 

In [1]:
import numpy as np
import matplotlib.pyplot as plt
import ipywidgets as widgets
from IPython.display import display

/home/patricia/.local/lib/python3.10/site-packages/matplotlib/projections/__init__.py:63: UserWarning: Unable to import Axes3D. This may be due to multiple versions of Matplotlib being installed (e.g. as a system package and as a pip package). As a result, the 3D projection is not available.
  warnings.warn("Unable to import Axes3D. This may be due to multiple versions of "


In [4]:
def euler_method(dt,init, start, end, fun):

    steps = int((end-start)/dt)
    vals = np.arange(start, end+dt, dt)
    vals[0] = init
    for ind, v in enumerate(vals[:-1]):
        vals[ind+1] = vals[ind] + dt * fun(vals[ind])

    return vals

def ODE(x):
    return 2 * x

start = 0
end = 1
init_x = 2

# Function that generates and plots data based on the slider value
def update_plot(value):

    t = np.arange(start, end+value, value)# Example function using the slider value
    x_approx = euler_method(value, init_x, start, end, ODE)

    plt.figure(figsize=(6, 4))
    plt.plot(t, init_x * np.exp(2*t),color = 'grey')
    plt.plot(t, x_approx, color = 'black', label = 'approximation')
    plt.plot([0,value],[init_x,x_approx[0]], color = 'red', label = 'dt')
    plt.xlabel('t')
    plt.ylabel('x')
    plt.xlim(start, end)
    plt.ylim(1.5, 15)
    plt.title('Euler integration step ='+str(value))
    plt.legend()
    plt.grid(True)
    plt.show()

# Create a slider
slider = widgets.FloatSlider(value=0.1, min=0.01, max=0.2, step=0.01, description="dt:")

# Use interactive to update the plot when the slider moves
interactive_plot = widgets.interactive_output(update_plot, {'value': slider})

# Display slider and interactive plot
display(slider, interactive_plot)

FloatSlider(value=0.1, description='dt:', max=0.2, min=0.001, step=0.001)

Output()

# Neuronal differential equations

In this workshop you will encounter two different types of neuron models. The rate model and the spiking neuron model. As mentioned before rate models can be treated analytically, spiking models can not. But to see the difference in the formulation, we will demonstrate how to simulate them both using the Euler method. 

The rate model describes neuronal activation using a single ODE which represents the firing rate. A typical rate neuron model is the following:

$\tau \frac{dr}{dt} = -r + I(t)$

where $I(t)$ is an external stimulus which can be turned on and off.

we can unpack the equation a bit more

Below you can find an example. Try and play around with the stimulus strength.

In [ ]:
def rate_neuron(r, I,tau = 10):
    return (-r + I)/tau

def integrate(neuron, time, stimulus, dt = 0.1):
    sim_time = int(time/dt)
    r_act = np.zeros([sim_time+1]) #init activation is 0 here
    t_record = np.zeros([sim_time])
    I = 0
    I_record =np.zeros([sim_time])

    for t in range(sim_time):
        if t > int(10/dt) and t < int(stimulus[1]/dt):
            I = stimulus[0]
        else:
            I = 0
        r_act[t+1] = r_act[t]+ dt * (rate_neuron(r_act[t],I))
        t_record[t] = dt * t
        I_record[t] = I

    return r_act[:-1], t_record, I_record


def update_rateplot(I_strength,I_duration ):

    rates, ts, stim = integrate(rate_neuron, 100, [I_strength,I_duration])#our time is in ms here


    plt.figure(figsize=(6, 4))
    plt.plot(ts, rates,label = 'r')
    plt.plot(ts, stim, color = 'grey', label = 'I')
    plt.xlabel('t[ms]')
    plt.ylabel('r')
    plt.title('Rate neuron')
    plt.legend()
    plt.grid(True)
    plt.show()

# Create a slider
slider = widgets.FloatSlider(value=10, min=0, max=100, step=1, description="strength:")
slider_l = widgets.FloatSlider(value=50, min=1, max=90, step=1, description="length[ms]:")

# Use interactive to update the plot when the slider moves
interactive_plot = widgets.interactive_output(update_rateplot, {'I_strength': slider,'I_duration': slider_l})

# Display slider and interactive plot
display(slider, slider_l, interactive_plot)

A spiking neuron model is chracterised by the addition of a spike generation mechanism. In a Leaky-Integrate and Fire (LIF) model this is implemented using a spike threshold and reset: If the membrane potential $v$ surpasses the defined spike threshold $\Theta_s$, the current membrane potential is set to the reset potential, e.g. to the resting potential $E_l$. This reset is a non-linearity as the membrane-potential after the reset does not linearly depend on its value prior to the reset. 

Mathematically the LIF neuron is defined as:

\begin{align*} \tau \frac{dv}{dt} = (E_l - v) + I(t) \\
if \ \ v(t) > \Theta_s: v(t) = E_l \end{align*}

where $v$ is the neuron's membrane potential, $E_l$ is the resting potential, $\tau$ is the membrane potential time constant, and $\Theta_s$ is the spiking threshold.

More complex version exists which includes differential equations for After-Spike reset currents, spike-threshold adaption, etc..

Below you can vary the received input to visualise its effect on the spiking rate and below threshold behaviour. 


In [ ]:
def LIF_neuron(v, I,Theta_s = -50, E_L = -70, tau = 10):
    if v < Theta_s:
        return ((E_L - v)+ I)/tau
    else:
        return E_L # immediate reset, we do not approximate the spike form here

def integrate(neuron, time, stimulus, dt = 0.1):
    sim_time = int(time/dt)
    v_act = np.zeros([sim_time+1])-70 # init activation is E_L here
    t_record = np.zeros([sim_time])
    I = 0
    I_record = np.zeros([sim_time])

    for t in range(sim_time):
        if t > int(10/dt) and t < int(stimulus[1]/dt): # define stimulus
            I = stimulus[0]
        else:
            I = 0
        v_act[t+1] = v_act[t]+ dt * (neuron(v_act[t], I)) # calculate potential for next time step
        t_record[t] = dt * t
        I_record[t] = I

    return v_act[:-1], t_record, I_record, np.nonzeros(vs >= -50)[0]


def update_rateplot(I_strength,I_duration ):

    vs, ts, stim,spikes = integrate(LIF_neuron, 100, [I_strength,I_duration])#our time is in ms here
 

    plt.figure(figsize=(6, 4))
    plt.plot(ts, vs,label = 'v')
    plt.scatter(spikes, spikes * 0 + -45, color = 'r')
    plt.plot(ts, stim, color = 'grey', label = 'I')
    plt.xlabel('t[ms]')
    plt.ylabel('v')
    plt.title('LIF neuron')
    plt.legend()
    plt.grid(True)
    plt.show()

# Create a slider
slider = widgets.FloatSlider(value=10, min=0, max=100, step=1, description="strength:")
slider_l = widgets.FloatSlider(value=50, min=1, max=90, step=1, description="length[ms]:")

# Use interactive to update the plot when the slider moves
interactive_plot = widgets.interactive_output(update_rateplot, {'I_strength': slider,'I_duration': slider_l})

# Display slider and interactive plot
display(slider, slider_l, interactive_plot)

# Noise

To drive neural activity one can either insert a constant external stimuli or noise into the neuron. Simulating noise has the advantage that it approximates the synaptic input from a large amount of neurons one does not need to model explicitly. 

The noisy current $\eta$ is commonly defined as a Ornstein-Uhlenbeck (OU) process. OU noise looks like a random walk. It integrates a Wiener process $\frac{dW}{dt}$ (time step adjusted white noise) overtime. It is defined as: 
\begin{align*}
\tau \frac{d\eta}{dt} = \theta \cdot (\mu - \eta) + \sigma \underbrace{\frac{dW}{dt}}_{\text{Wiener process}} 
\end{align*}
 with $\mu$ is the mean value around which the process fluctuates, $\theta$ is the drive towards the mean and $\sigma$ the spread of the process. The spread defines how strongly the noise fluctuates around its mean. The bigger $\sigma$ the higher are the deviations from the $\mu$. The time constant $\tau$ defines the fluctuation speed. The higher the the time constant, the slower the fluctuations. The drive $\theta$ determines how fast the OU process approches $\mu$ after initialisation. 

In practice we simulate this noise process using the Euler method, which gives the following equation:
\begin{align*}
\eta(t+1) = \eta(t) + \frac{dt}{\tau} \theta \cdot (\mu - \eta) + \sigma  \underbrace{\sqrt{\frac{2dt}{\tau}}\cdot N(0,1)}_{\text{Wiener process}}
\end{align*}

The scaling of the Wiener process is necessary to ensure independence of the spread $\sigma$ and the integration time step.

Below you can try how the activity of a rate neuron and a LIF (same formulations as above) differ depending on the standard deviation and mean of the OU noise. ($ \theta = 1$ and therefore omitted in the code.)

In [ ]:
rng = np.random.RandomState(seed = 5) #commentar zum random seed, eventuell wegnehmen

def integrate(time, stimulus, dt = 0.1, tau = 20):
    rng = np.random.RandomState(seed = 5)
    sim_time = int(time/dt)
    v_act = np.zeros([sim_time+1])-70 #init activation is E_L here
    r_act = np.zeros([sim_time+1])
    t_record = np.zeros([sim_time])
    I =np.zeros([sim_time+1])+stimulus[0] #we start at the mean value so we do not need a warm-up period to get the statistics correct

    for t in range(sim_time):
        I[t+1] = I[t] + dt /tau*(stimulus[0] - I[t]) + np.sqrt(2*dt/tau) * stimulus[1] * rng.randn()
        v_act[t+1] = v_act[t]+ dt * (LIF_neuron(v_act[t],I[t]))
        r_act[t+1] = r_act[t]+ dt * (rate_neuron(r_act[t],I[t]))
        t_record[t] = dt * t

    return v_act[:-1], r_act[:-1], t_record, I[:-1]


def update_noiseplot(mean,sd ):

    vs, rs, ts, stim = integrate( 100, [mean,sd])#our time is in ms here


    fig, ax = plt.subplots(3,1,figsize=(6, 6))
    ax[0].plot(ts, stim)
    ax[0].plot(ts, stim*0+mean, color = 'grey', linestyle = ':')
    ax[0].set_xlabel('t[ms]')
    ax[0].set_ylabel('$\eta$')
    ax[0].set_title('OU Noise')
    ax[0].grid(True)

    ax[1].plot(ts,rs)
    ax[1].plot(ts, rs*0, color = 'grey', linestyle = ':')
    ax[1].set_xlabel('t[ms]')
    ax[1].set_ylabel('$r$')
    ax[1].set_title('Rate Neuron')
    ax[1].grid(True)

    ax[2].plot(ts, vs)
    ax[2].plot(ts, vs*0-70, color = 'grey', linestyle = ':')
    ax[2].set_xlabel('t[ms]')
    ax[2].set_ylabel('$v$')
    ax[2].set_title('LIF Neuron')
    ax[2].grid(True)

    fig.tight_layout()

# Create a slider
slider = widgets.FloatSlider(value=0, min=-20, max=100, step=1, description="μ:")
slider_l = widgets.FloatSlider(value=2, min=0.1, max=20, step=.1, description="σ:")

# Use interactive to update the plot when the slider moves
interactive_plot = widgets.interactive_output(update_noiseplot, {'mean': slider,'sd': slider_l})

# Display slider and interactive plot
display(slider, slider_l, interactive_plot)

# Simulation modules & packages

So far, we programmed every detail of the system ourselfes. However, this is strictly not neccessary. There are plenty of python packages and modules specifically dedicated to simulating dynamical systems and networks. This means that you do not have to take care about any implementational details. For most simulation softwares, you only need to define your network. The integration method can be specified, but is implemented effeiciently in the back-end. This enables us to simualte larde networks without having to increase computationally effeciency of our code, as the packages are already tuned for this. Using simulation modules can help to mitigate some of the complexity-computational-cost trade-off. This, finding the right package can help you speed up your simulations significantly: **Neuron**, **Nest**, **BRIAN**, **Dendrify** to name a few. 


In the following, we will show you how to implement the above noise neuron with **BRIAN**. During the Spiking Neural Network tutorial we will continue using that package. It has an intuitive syntax which makes network set-up easy to follow.

In [ ]:
from brian2 import *
from brian2tools import *

To define a model in **BRIAN**, you have to specify it as a differential equation of the form: 

    differential_equation = ''' dx/dt = dif(x) : unit'''  

The *unit* is always given in SI units. **BIRAN** checks your differential equation for unit compatibility. This means that you will get an error message if a differential equation specifying the behaviour of the membrane potential of a neuron does not conform with the units of $v/s$ or if you try to add a variable measured in Hertz to a variable measured in Ampere. This is specifically helpful when having to convert between units. 

However, you do not have to specify units. You can also define dimensionless variables with a unit of $1$. 
You can add neural attributes by specifying the variable after the differntial equation:

    '''variable:unit'''

For example you can specify the membrane time constant that way and give each neuron in your network a different time constant by setting it explicitly. Below you fined the definitions for a dimensionless rate neuron, a rate neuron which activity is measured as a rate explicitly and a LIF neuron. Any variable which is dependent on t like *I_rate(t)*, *I[t]* and *I_spike(t)* are external time-dependent input. Their activity is not part of the neuron definition but is set individually. They can be used to insert a square stimuli into the neuron without having to stop the simulation, or to present repeating stimuli patterns. 

In [ ]:
#Differential equations
#dimensionless rate neuron
diff_eq_rate = '''dr/dt = (-r + I_rate(t))/tau : 1
               tau : second'''
#Frequency rate neuron
diff_eq_rate_Hz = '''dr/dt = (-r + I[t])/tau : Hz
               tau : second'''
#LIF neuron
diff_eq_spike = '''dv/dt = ((E_L - v) + I_spike(t))/tau : volt (unless refractory)
                   E_L : volt
                   v_spike : volt
                   tau : second'''





def run_brian_neurons(length, strength):
    start_scope() #to keep the networks clean when running the same entities multiple times
    
    #Define the neuron. Syntax: NeuronGroup(number of neurons, differential equation)
    #we explicitly specify to use the euler method
    N_rate = NeuronGroup(1, diff_eq_rate, method = 'euler')
    N_rate.tau = 10 * ms
    N_spike = NeuronGroup(1, diff_eq_spike, threshold = 'v > v_spike', reset = 'v = E_L', refractory = 3 * ms, method = 'euler') #our spiking neuron has a refactory period of 3ms
    #the LIf neuron also has a refractory period during which we stop integration
    N_spike.E_L = -79 * mV
    N_spike.v_spike = -49 * mV
    N_spike.v = -79 * mV  #we need define v(0) otherwise it is assumed to be 0
    N_spike.tau = 10 * ms

    #we know define the time-dependent input
    input_arr = np.zeros([200]) #the stimulus length is 200ms
    onset = 10 #our stimulus begins after 10ms
    input_arr[onset:int(length)+onset] = strength
    I_rate = TimedArray(input_arr, dt = 1*ms) #each entry in the TimedArray lasts for 1ms
    I_spike = TimedArray(input_arr * mV, dt = 1*ms) #needs to be in mV to be compatible with dimensions of the model (with proper consideration of conductances we could also inject a current here)

    #to record activity we need to define monitors
    s_v = SpikeMonitor(N_spike)
    
    #Syntax: StateMonitor(group to record from, variable to record, which neurons to record from, recording frequency)
    record_r = StateMonitor(N_rate, ['r'], record = True, dt = 1 * ms) #record = True means we record from the full neurongroup
    record_v = StateMonitor(N_spike, ['v'], record = True, dt = 1 * ms)

    run(500 * ms, report = 'text') #run the network for 500ms, you can delete the status report by omitting the second argument in the function

    return s_v, record_r, record_v



def update_brianplot(stim_l, stim_s ):

    spikemon, rec_r, rec_v = run_brian_neurons(stim_l, stim_s)

    fig, ax = plt.subplots(2,1,figsize=(6, 6))
    ax[0].plot(rec_r.t, rec_r.r[0])
    ax[0].set_xlabel('t[ms]')
    ax[0].set_ylabel('$r$')
    ax[0].set_title('Rate Neuron')
    ax[0].grid(True)

    ax[1].plot(rec_v.t,rec_v.v[0]/mV)
    ax[1].scatter(spikemon.t, spikemon.i - 45, color = 'r') #we set the indicators for the spike a bit above the spike threshold. if you want to plot a spike raster you just use SpikeMonitor.i
    ax[1].set_xlabel('t[ms]')
    ax[1].set_ylabel('$v[mV]$')
    ax[1].set_title('LIF Neuron')
    ax[1].grid(True)

    fig.tight_layout()

# Create a slider
slider = widgets.FloatSlider(value=10, min=0, max=100, step=1, description="duration:")
slider_l = widgets.FloatSlider(value=50, min=0, max=100, step=.1, description="amplitude:")

# Use interactive to update the plot when the slider moves
interactive_plot = widgets.interactive_output(update_brianplot, {'stim_l': slider,'stim_s': slider_l})

# Display slider and interactive plot
display(slider, slider_l, interactive_plot)